In [4]:
import pandas as pd

### Read datasets

In [5]:
#reading datasets
df_mailing = pd.read_excel('2 residential exemption mailing.xlsx', sheetname='Sheet1')
df_mailing = df_mailing.drop(columns=['Unnamed: 2','Unnamed: 9','Unnamed: 13'])
df_rk1 = pd.read_excel('4 All Residential data_rk.xlsx', sheetname='Assessors')
df_rk2 = pd.read_excel('4 All Residential data_rk.xlsx', sheetname='Resident List')
df_rk1.rename(columns=lambda x:x.replace(' ','_'), inplace=True)
df_rk2.rename(columns=lambda x:x.replace(' ','_'), inplace=True)
df_mailing.rename(columns=lambda x:x.replace(' ','_'), inplace=True)

/Users/jxtang/anaconda3/lib/python3.6/site-packages/pandas/util/_decorators.py:118: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


In [6]:
df_mismatch=[]
total0 = 0
for row in df_rk1.itertuples():
    if row.Address_Match == False:
        df_mismatch.append(row)
        total0 += 1
    if row.Address_Match == True:
        total0 += 1
df_mismatch = pd.DataFrame(df_mismatch)

### data cleaning functions

In [7]:
#data cleaning
def upperletter(row):
    return row.str.upper()
def trimdata(row):
    return row.str.strip()
df_rk1[['Merged','Mailing_Address','Owner_1','Owner_2','Owner_3']] = df_rk1[['Merged','Mailing_Address','Owner_1','Owner_2','Owner_3']].apply(trimdata, axis=1).apply(upperletter, axis=1)
df_rk2[['Merged','Street_name','Name']] = df_rk2[['Merged','Street_name','Name']].apply(trimdata, axis=1).apply(upperletter, axis=1)
df_mailing = df_mailing.apply(trimdata, axis=1)

In [8]:
#mailing.bedroom.unit dataset
df_mbu = pd.read_excel('mailing.bedroom.unit.xlsx')
df_mbu.rename(columns=lambda x:x.replace(' ','_'), inplace=True)
df_mbu = df_mbu.drop(columns=['Unnamed:_2','Unnamed:_9','Unnamed:_13'])
df_mbu[['Mailing_Street_Address','Owner','Second_Owner','Third_Owner']] = df_mbu[['Mailing_Street_Address','Owner','Second_Owner','Third_Owner']].apply(trimdata, axis=1)
df_mbu[['Mailing_Street_Address','Owner','Second_Owner','Third_Owner']] = df_mbu[['Mailing_Street_Address','Owner','Second_Owner','Third_Owner']].apply(upperletter, axis=1)

### calculate address mismatch by mailing address

In [9]:
# print(df_rk1.shape)
# print(df_mismatch.shape)
# print(df_mismatch.columns)
# print(df_rk1.columns)
print("mailing mismatch percent:",df_mismatch.shape[0]/total0)
merged = set(df_rk2.Name)
df_extract = []
rv = 0
for row in df_rk1.itertuples():
    if row.Owner_1 in merged or row.Owner_2 in merged or row.Owner_3 in merged:
        df_extract.append(row)
    if row.Mailing_City == 'REVERE ':
        rv += 1
df_extract = pd.DataFrame(df_extract) 
rv

mailing mismatch percent: 0.26574181757767074


8965

In [10]:
#extract dataframe that address exist in both sheets.
merged2 = set(df_rk2.Merged)
tmp = []
for row in df_rk1.itertuples():
    if row.Merged in merged2 or row.Mailing_Address in merged2:
        tmp.append(row)
df_extract2 = pd.DataFrame(tmp)

In [11]:
#create a dict to store all the owners that have multiple properties based on df_rk2.
#create a list to store the owner names.
#or it may result in duplicate names.
addr_dict={}
multi_owner = []
multi = 0
for row in df_rk2.itertuples():
    Merged2 = str(row.House_number)+' '+row.Street_name
    if row.Name not in addr_dict:
        addr_dict[row.Name] = [Merged2]
    else:
        if [Merged2] == addr_dict[row.Name]:
            continue
        else:
            addr_dict[row.Name].append(Merged2)
            multi_owner.append(row.Name)

In [12]:
len(addr_dict)

36609

In [13]:
#measure the percentage by person
comp_names = []
num = 0
all_addr = []
noop_addr = []
for row in df_rk1.itertuples():
    if row.Owner_1 in addr_dict or row.Owner_2 in addr_dict or row.Owner_3 in addr_dict:
        all_addr.append(row.Mailing_Address)
    if row.Owner_1 in addr_dict:
        num+=1
        if row.Mailing_Address not in addr_dict[row.Owner_1]:
            comp_names.append(row.Owner_1)
            noop_addr.append(row.Mailing_Address)
    if row.Owner_2 in addr_dict:
        num+=1
        if row.Mailing_Address not in addr_dict[row.Owner_2]:
            comp_names.append(row.Owner_2)
            noop_addr.append(row.Mailing_Address)
    if row.Owner_3 in addr_dict:
        num+=1
        if row.Mailing_Address not in addr_dict[row.Owner_3]:
            comp_names.append(row.Owner_3)
            noop_addr.append(row.Mailing_Address)
m_owners = set(comp_names)
all_addr = set(all_addr)
noop_addr = set(noop_addr)
print('names with different address:',len(m_owners))
print('names appear in both sheets:',num)
print('percent that a person have multiple properties:',len(m_owners)/num)
print(len(all_addr))
print(len(noop_addr))
print(len(noop_addr)/len(all_addr))

names with different address: 623
names appear in both sheets: 4136
percent that a person have multiple properties: 0.15062862669245647
3044
568
0.18659658344283836


In [14]:
#data comparison: add the new mailing.bedroom.unit dataset
comp_names2 = []
num2 = 0
noop_addr2 = []
for row in df_mbu.itertuples():
#     print(row.Owner)
    if row.Owner in addr_dict:
        num2+=1
        if row.Mailing_Street_Address not in addr_dict[row.Owner]:
            comp_names2.append(row.Owner)
            noop_addr2.append(row.Mailing_Street_Address)
    if row.Second_Owner in addr_dict:
        num2+=1
        if row.Mailing_Street_Address not in addr_dict[row.Second_Owner]:
            comp_names2.append(row.Second_Owner)
            noop_addr2.append(row.Mailing_Street_Address)
    if row.Third_Owner in addr_dict:
        num2+=1
        if row.Mailing_Street_Address not in addr_dict[row.Third_Owner]:
            comp_names2.append(row.Third_Owner)
            noop_addr2.append(row.Mailing_Street_Address)

mm_owners = set(comp_names2)
print('names with different address new:',len(mm_owners))
print('names appear in both sheets new:',num2)
print('percent that a person have multiple properties:',len(mm_owners)/num2)
noop_addr2 = set(noop_addr2)
print(len(noop_addr2))

names with different address new: 655
names appear in both sheets new: 3705
percent that a person have multiple properties: 0.1767881241565452
582


In [15]:
len((noop_addr2)&(noop_addr))

423

In [16]:
len(noop_addr2)

582

In [17]:
len(noop_addr|noop_addr2)

727

In [18]:
noop_addr_merge = (noop_addr|noop_addr2)

In [19]:
len(noop_addr_merge)

727

In [20]:
#### new methods

In [21]:
df_as = pd.read_excel('Modify_ASRL.xlsx', sheetname='Modify_AS')
df_rl = pd.read_excel('Modify_ASRL.xlsx', sheetname='Modify_RL')

/Users/jxtang/anaconda3/lib/python3.6/site-packages/pandas/util/_decorators.py:118: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


In [22]:
sname_rl = []
name_rl = []
def trim2(row):
    return row.replace('-','')
df_rl['Sorted_Owner'].apply(trim2)
for row in df_rl.itertuples():
    sname_rl.append(row.Sorted_Owner)
    name_rl.append(row.Name)

In [23]:
sname_rl
name_rl

['HORROCKS RUTH E',
 'MACANCELA SONIA',
 'MACANCELA JHONNY OSWALDO',
 'SANTACRUZ KATHERINE JANETH',
 'MACANCELA KELLY N',
 'MACANCELA CRISTOPHER A',
 'HERNANDEZ ROBERTO A',
 'OCASIO SALVADORA',
 'HERNANDEZ ROBERTO A',
 'BARRETO EDMARIET',
 'HERNANDEZ SONYA E',
 'SAENZ JAMIE',
 'SAENZ JUANCARLOS',
 'SAENZ MARTHA L',
 'BELMONTE DEBORAH',
 'BELMONTE ANTHONY A',
 'ELDEEB AMIR K',
 'ABDOU NISRINE',
 'ABDOU ILYASS',
 'SOLEMINA REGIS R',
 'PEREIRA MARCIO I',
 'PEREIRA JUSSANE',
 'OLIVEIRA CILEIMAR S',
 'COUTINHO ISABEL O',
 'SILVA GILSON',
 'COUTINHO ALINE OLIVEIRA',
 'SILVA JENNA S',
 'VENTURA JAEDER',
 'EBEJA ARMAND',
 'POWERS DOLORES',
 'AGUILAR MARLENY',
 'VELASQUEZ ARIEL O',
 'PERRETTI DOMENIC A',
 'PERRETTI LORETTA',
 'PERRETTI COREY J',
 'ORREGO NEBARDO',
 'SHELDON RICHARD W',
 'ORREGO MALLORY',
 'BERNARD LLEANNA G',
 'WAPLE JENNIFER M',
 'RIVERA HECTOR A',
 'RIVERA SUSAN',
 'CARSON ZACKARY RYAN',
 'RIVERA ALEXANDER',
 'ORTEZ ENNA R',
 'GUTIERREZ DANIEL A',
 'GUTIERREZ CECY',
 'BRANGIF

In [24]:
sname_as = []
name_as = []
for row in df_as.itertuples():
    for i in row.Sorted_Owner:
        pass
        #print(i)
        #sname_as.append(names)
    name_as.append(row.Owner_1)
    name_as.append(row.Owner_2)
    name_as.append(row.Owner_3)